# Train Classifier

## Libraries

In [1]:
import os, json
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # or any {'0', '1', '2','3'}
from shutil import rmtree


from tensorflow import keras
from keras import applications
from keras import callbacks


##  Definitions

In [2]:
v_workspace='./exp_Xception'
v_path_dataset='../data/agriculture/rice_image_dataset'
v_input_shape=(224, 224, 3)
v_weights='imagenet'

v_batch_size=64 
v_epochs = 50

v_metrics=['MeanSquaredError','AUC','Precision','Recall','accuracy']


v_image_size=(v_input_shape[0],v_input_shape[1])
v_color_mode="grayscale" if v_input_shape[2]==1 else "rgb" # "rbga"
if v_color_mode=="grayscale" and v_weights=='imagenet':
    v_weights=None
    print ("weights set to null as color mode is grayscale")

## Functions

In [3]:
# Dataset
def load_dataset(root_dataset='../data/sampledataset', image_size=(299,299),color_mode='grayscale', batch_size=16 ):
    datagen = keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=True,
        featurewise_std_normalization=False,
        samplewise_std_normalization=True,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.0,
        height_shift_range=0.0,
        brightness_range=None,
        shear_range=0.0,
        zoom_range=(0.95,0.95),#####
        channel_shift_range=0.0,
        fill_mode='nearest',
        cval=0.0,
        horizontal_flip=False,
        vertical_flip=False,
        rescale=1./255,#######
        preprocessing_function=None,
        data_format=None,# 'channels_last'
        validation_split=0.2,
        dtype=None,#tf.float32
    )

    ds_train = datagen.flow_from_directory(
        directory=os.path.join (root_dataset),
        target_size=image_size,
        color_mode=color_mode, #"rgb",#grayscale', #
        classes=None,# 'sparse' for integer based instead of hot-encode
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=101,
        save_to_dir=None,
        save_prefix='',
        save_format='png',
        follow_links=False,
        subset='training',
        interpolation='nearest'
    )

    ds_validate = datagen.flow_from_directory(
        directory=os.path.join (root_dataset),
        target_size=image_size,
        color_mode=color_mode, #"rgb",#grayscale', #
        classes=None,# 'sparse' for integer based instead of hot-encode
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=101,
        save_to_dir=None,
        save_prefix='',
        save_format='png',
        follow_links=False,
        subset='validation',
        interpolation='nearest',
    )

    #class_names=ds_train.class_indices
    #print (class_names)
    #class_names = list(class_names.keys())
    return (ds_train, ds_train.labels),(ds_validate,ds_validate.labels)

In [4]:
def save_class_indices(class_indices_json,path_to_workspace):
    file_path=os.path.join(path_to_workspace,'class_indices.json')
    class_indices_json = dict((v,k) for k,v in class_indices_json.items())
    print (class_indices_json)
    with open(file_path, "w") as write_file:
        json.dump(class_indices_json, write_file)

In [5]:
rmtree(v_workspace) if os.path.isdir(v_workspace) else None
os.mkdir(v_workspace) 

#os.mkdir(v_workspace) if not(os.path.isdir(v_workspace)) else rmtree(v_workspace)

path_csvname = os.path.join(v_workspace,'performance.csv')
path_tensorboardLog= os.path.join(v_workspace,'tensorboard')
path_checkpoint=os.path.join(v_workspace,'checkpoint')

                            
CSVLogger_cb = callbacks.CSVLogger(path_csvname,  separator=',',   append=True)


TensorBoard_cb = callbacks.TensorBoard(log_dir=path_tensorboardLog,
                                          histogram_freq=1,
                                          write_graph=True,
                                          write_images=True,
                                          update_freq=v_batch_size,
                                          #write_steps_per_second=False,
                                          profile_batch=2,
                                          embeddings_metadata=None)


ModelCheckpoint_cb = callbacks.ModelCheckpoint(filepath=path_checkpoint,
                                                save_best_only=True, ###to save space
                                                save_weights_only=False,
                                                monitor='val_accuracy',
                                                mode='max')


EarlyStopping_cb = callbacks.EarlyStopping(monitor='val_loss', 
                                                   min_delta=0.001,
                                                   patience=6,
                                                   verbose=0, 
                                                   mode='auto',
                                                   baseline=None,
                                                   restore_best_weights=False)

TerminateOnNaN_cb = callbacks.TerminateOnNaN()

ReduceLROnPlateau_cb = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                            factor=0.01,
                                                            patience=5,
                                                            verbose=0,
                                                            mode='auto',
                                                            min_delta=0.001,
                                                            cooldown=0,
                                                            min_lr=0)

def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr * tf.math.exp(-0.0001)

LearningRateScheduler_cb = callbacks.LearningRateScheduler(scheduler, 
                                                                    verbose=0)



RemoteMonitor_cb = callbacks.RemoteMonitor(root='http://localhost:9000',
                                                    path='/publish/epoch/end/',
                                                    field='data',
                                                    headers=None,
                                                    send_as_json=False)

## Data

In [6]:
(x_train, y_train), (x_valid, y_valid)=load_dataset(root_dataset=v_path_dataset, 
                                                    image_size=v_image_size,
                                                    color_mode=v_color_mode, 
                                                    batch_size=v_batch_size )
num_of_classes=x_train.num_classes
class_indices=x_train.class_indices

Found 60000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.


In [7]:
save_class_indices(class_indices_json=class_indices,path_to_workspace=v_workspace)

{0: 'Arborio', 1: 'Basmati', 2: 'Ipsala', 3: 'Jasmine', 4: 'Karacadag'}


## Modeling

In [8]:
# https://lightrun.com/answers/keisen-tf-keras-vis-graph-disconected--transfer-learning-model-with--top-layer
def get_model():
    
    '''
    applications.inception_v3.InceptionV3
    applications.vgg16.VGG16
    applications.xception.Xception
    applications.mobilenet_v2.MobileNetV2

    '''
    base_model = applications.xception.Xception(
        weights=v_weights,  # Load weights pre-trained on ImageNet.
        input_shape=v_input_shape,
        include_top=False) 
    base_model.trainable = False
    
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    #x = keras.layers.Dense(num_of_classes, activation='softmax')(x)
    x = keras.layers.Dense(num_of_classes)(x)
    outputs= keras.layers.Softmax()(x)
    model = keras.Model(base_model.inputs, outputs)
    return model
model=get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

## Training

In [9]:
# compile
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=v_metrics
              )

In [ ]:
#Train the top layer
callbacksList=[CSVLogger_cb,TensorBoard_cb,ModelCheckpoint_cb,EarlyStopping_cb,TerminateOnNaN_cb,ReduceLROnPlateau_cb]
print ("Training starting...\nTo see result with tensorboard, execute the following in separate terminal:")
print (\t"tensorboard --logdir="+v_workspace+"/tensorboard --host 0.0.0.0 --port 6006\n\n")
history=model.fit(x=x_train,
                  validation_data=(x_valid),
                  batch_size=v_batch_size,
                  epochs=v_epochs, 
                  callbacks=callbacksList,
                  shuffle=True
                 )


Training starting...
To see result with tensorboard, execute the following in separate terminal:
	
tensorboard --logdir=./exp_Xception/tensorboard --host 0.0.0.0 --port 6006


Epoch 1/50
444/938 [=============>................] - ETA: 5:14 - loss: 0.3787 - mean_squared_error: 0.0388 - auc: 0.9781 - precision: 0.9247 - recall: 0.8022 - accuracy: 0.8719

## Save

In [ ]:
fname = os.path.join (v_workspace,"saved_model")
savedModel = os.path.join(v_workspace, "saved_model")
savedWeights = os.path.join(v_workspace,"saved_weights","weights")

model.save(savedModel)
model.save_weights(savedWeights)